In [4]:
import pandas as pd

mints_df = pd.read_csv('mints.csv', low_memory=False)
rugpull_labels_df = pd.read_csv('rugpulls_with_token_info.csv', low_memory=False)

# Ensure that 'id' is the correct identifier for pools
print(mints_df.head())

# Convert 'date' to a datetime object
mints_df['timestamp'] = pd.to_datetime(mints_df['timestamp'], unit='s')


                    amount       amount0     amount1    amountUSD  \
0    613460599434217667567  1.938998e+03    0.426067  3319.199673   
1    891849916738551272778  0.000000e+00    2.000000  6313.274661   
2  17701992740182515804492  1.636746e+04    1.811366  2780.160697   
3     60361793806163010168  1.445108e+05    0.000073     0.162941   
4  48928544071541478942235  2.394002e+07  100.000000     0.000000   

                                                  id  logIndex  \
0  0x0000118dcfb5178f1776acb8d6c3f1229eb84475c5c5...       176   
1  0x00001a3315046ff0782b5becdad6a43855d1e5a0e683...        97   
2  0x00001c5d54ad8669ec30ac71a4e8d7eae7096bb4fae5...       641   
3  0x0000205d7b543532a515537a0f70e6eb63103ebd80ec...       471   
4  0x000030188de410a6ad662a97e1b0ba2b1e9868863b80...       214   

                                       origin  \
0  0xadc8aaddf5dcf81366fcc0212c154dfbfde1ed13   
1  0x57db5d6aa783cf29af41330569d24957140fd3eb   
2  0x99dd497521cdd8371daca3314c89864dc21b

In [14]:
import pandas as pd

# Initialize a list to store the feature rows
feature_rows = []
# Initialize an empty DataFrame for mint features
mint_features = pd.DataFrame()

import numpy as np

count = 0
# Initialize a list to store the feature rows
feature_rows = []
# Initialize a DataFrame to store the features
pool_features = pd.DataFrame()

for index, pool_id in enumerate(rugpull_labels_df['pool_id']):
    if index % 200 == 0:
        print(f"Processing pool {index}: Pool ID {pool_id}")
        print(f"Number of pools skipped: {count}")
        # if index > 0: break

    rugpull_date = rugpull_labels_df[rugpull_labels_df['pool_id'] == pool_id]['date'].iloc[0] # Get the rugpull date for this pool

    token0_symbol = rugpull_labels_df[rugpull_labels_df['pool_id'] == pool_id]['token0.symbol'].iloc[0]
    token1_symbol = rugpull_labels_df[rugpull_labels_df['pool_id'] == pool_id]['token1.symbol'].iloc[0]

    # print(rugpull_date)
    mint_data = mints_df[mints_df['pool.id'] == pool_id]

    # if (len(mint_data) < 3):
    #     # print(f"Skipping pool {pool_id} because there is no data")
    #     count += 1
    #     continue

    # If rugpull_date is NaT, use all available data; else filter data before rugpull
    pre_rugpull_data = mint_data if pd.isna(rugpull_date) else mint_data[mint_data['timestamp'] < rugpull_date]
    
    if pre_rugpull_data.empty:
        print(f"Skipping pool {pool_id} because there is no data before the rugpull date")
        continue

    # # Calculate the distribution of minted tokens for the top 10 addresses
    # grouped_owners = pre_rugpull_data.groupby('origin')['amount'].sum()
    # print(grouped_owners[0])
    # top_owners_amount = grouped_owners.nlargest(10).sum()
    # total_minted_amount = grouped_owners.sum()
    # distribution_top_10 = top_owners_amount / total_minted_amount if total_minted_amount != 0 else 0

    # total_minted_amount = pre_rugpull_data['amount'].sum()
    # distribution_top_10 = top_owners_amount / total_minted_amount if total_minted_amount != 0 else 0
    
    pre_rugpull_data = pre_rugpull_data.select_dtypes(include=[np.number])
    
    # Identify if token0 or token1 is WETH and rename columns
    if token0_symbol == 'WETH':
        weth_token = 'amount0'
        otherToken = 'amount1'
    elif token1_symbol == 'WETH':
        weth_token = 'amount1'
        otherToken = 'amount0'
    else:
        print(f"Skipping pool {pool_id} because neither token is WETH")
        continue

    # Rename columns
    pre_rugpull_data.columns = [col.lower().replace(weth_token, 'wethToken').replace(otherToken, 'otherToken') for col in pre_rugpull_data.columns]

    # # Debugging: Print new column names
    # print("New Column Names:", pre_rugpull_data.columns)
    
    # Calculate mean and variance for relevant columns
    means = pre_rugpull_data.mean().fillna(0)
    variances = pre_rugpull_data.var().fillna(0)  # Replacing NaN with 0

    feature_row = {}

    feature_row = {'pool_id': pool_id}
    for mean in means.index:
        feature_row[f'mean_{mean}'] = means[mean]
    for variance in variances.index:
        feature_row[f'variance_{variance}'] = variances[variance]
        
    feature_rows.append(feature_row)

# Convert the list of dictionaries to a DataFrame
pool_features = pd.DataFrame(feature_rows)

print(pool_features.head())


Processing pool 0: Pool ID 0x000c0d31f6b7cecde4645eef0c4ec6a492659d62
Number of pools skipped: 0
Processing pool 200: Pool ID 0x0a1665e3f54eeb364bec6954e4497dd802840a01
Number of pools skipped: 0
Processing pool 400: Pool ID 0x152ac8b0358a215fe1f6dd75c8804e4f6ca046eb
Number of pools skipped: 0
Processing pool 600: Pool ID 0x1f138debc0721b364a967b09402b45069cba7b35
Number of pools skipped: 0
Processing pool 800: Pool ID 0x291dae7ebf6193910bf69cb14f121c07cfea4de0
Number of pools skipped: 0
Processing pool 1000: Pool ID 0x3429e990337542434f6bfac5a4d9f14ed285ac7a
Number of pools skipped: 0
Processing pool 1200: Pool ID 0x3e441eb04d28e1f8cac30f6e32d736a96d8a13b1
Number of pools skipped: 0
Processing pool 1400: Pool ID 0x46ca72148708fbc450e046fffd264a7821f70ba3
Number of pools skipped: 0


In [ ]:
pool_data_with_labels = pd.read_csv('pool_feature_data_with_labels.csv', low_memory=False)

# Merge the DataFrames on the 'pool_id' column
merged_data = pool_data_with_labels.merge(pool_features, on='pool_id', how='left')

# Save the merged DataFrame to a new CSV file, if needed
merged_data.to_csv('merged_pool_data.csv', index=False)

print("Merged data saved to merged_pool_data.csv")